<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema2_Informe2_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Análisis de la relación entre variación del PIB y muertes por COVID19 en los países europeos de la OCDE

Fecha del informe: 24/10/2022.

Autor: Daniel Oto Peralías.

In [ ]:
# Importar librerías 
import pandas as pd
pd.set_option('display.max_row', 40)
pd.set_option('display.max_columns', 10)
import matplotlib.pyplot as plt

In [ ]:
# Subir datos del PIB
from google.colab import files
files.upload()

In [ ]:
# Leer datos de variación del PIB
pib=pd.read_excel('datosOCDE2022.xlsx')
# Seleccionamos columnas relevantes
pib=pib[["Country","2020","2021"]]

In [ ]:
# Creamos una lista con los países de la OCDE
paises=pib.Country.to_list()

In [ ]:
# Datos COVID (contiene solo países europeos)
covid = pd.read_csv("https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/owid-covid-data_eu.csv",sep=",", index_col=0, parse_dates=True)

In [ ]:
# Filtro para quedarme solo con los países de la OCDE
# Primero creo una variable =1 si el país pertenece a la OCDE
covid['ocde']=0
for x in paises:
  covid.loc[covid['location']==x,'ocde']=1

In [ ]:
# Como solamente tengo países europeos en la tabla de COVID, debo tener un total de 26 países donde "ocde"==1
len(covid.loc[covid['ocde']==1,"location"].unique())

In [ ]:
# Veo que faltan 2. 

# Para ver los países que faltan
paises_identif=covid.loc[covid['ocde']==1,'location'].unique()
for x in paises:
    if x not in paises_identif:
        print(x)

In [ ]:
# De los anteriores, europeos son solo Czech Republic y Slovak Republic

# Cambiamos el nombre en tabla covid
covid.loc[covid['location']=='Czechia',"location"]='Czech Republic'
covid.loc[covid['location']=='Slovakia',"location"]='Slovak Republic'

In [ ]:
# Volvemos a seleccionar los países de la OCDE
covid['ocde']=0
for x in paises:
  covid.loc[covid['location']==x,'ocde']=1
len(covid.loc[covid['ocde']==1,"location"].unique()) # Bien! Ahora están los 26

In [ ]:
# Ahora seleccionamos la muestra de países de la OCDE
covid=covid.loc[covid['ocde']==1]

In [ ]:
# Creamos la tabla en formato deseado: 
  # una fila por país 
  # y dos columnas, una para muertes acumuladas en 2020 y otra para muertes acumuladas durante 2021

# Primero creo una tabla con el total de muertes a 31/12/2020 (== muertes acumuladas 2020)
covid2020=covid.loc[covid.index=="2020-12-31",["location","total_deaths_per_million"]].copy()

# Y otra con el total de muertes a 31/12/2021 (== muertes acumuladas durante 2020 y 2021)
covid2021=covid.loc[covid.index=="2021-12-31",["location","total_deaths_per_million"]].copy()

# Mezclo ambas tablas
muertescov=covid2020.merge(covid2021,on="location")
muertescov

In [ ]:
# Cambio el nombre de las columnas
muertescov.rename(columns={"location":"Country","total_deaths_per_million_x":"muertes2020","total_deaths_per_million_y":"muertes2020_21"},inplace=True)
muertescov

In [ ]:
# Creamos muertes2021: las muertes acumuladas DURANTE 2021
muertescov['muertes2021']=muertescov.muertes2020_21-muertescov.muertes2020
muertescov

In [ ]:
# Unimos tabla PIB con tabla muertescov
df=pib.merge(muertescov,on="Country")
df

In [ ]:
df.set_index("Country",inplace=True)

In [ ]:
# Crear gráfico de barras
pib.sort_values('2020', inplace=True) # Idem
pib.plot.bar()

In [ ]:
# Crear lista de países de la OCDE para luego seleccionarlos en OWID
paises=pib.index.to_list()

In [ ]:
# Opción 1: Importar datos de OWID de COVID19 (LEYÉNDOLOS DIRECTAMENTE)
datapath="https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/owid_covid.xlsx"
covid=pd.read_excel(datapath,index_col=0,parse_dates=True)

In [ ]:
# Opción 2: 2.1 Subir datos
files.upload()

In [ ]:
# Opción 2: 2.2 Leer datos
covid=pd.read_excel("/content/owid_covid.xlsx",index_col=0,parse_dates=True)

In [ ]:
cov.set_index("location", inplace=True)

In [ ]:
# Gráfico exploratorio de barras
cov.sort_values("total_deaths_per_million").total_deaths_per_million.plot.bar()

In [ ]:
pib.reset_index(inplace=True)
cov.reset_index(inplace=True)
cov.rename(columns={"location":"Country"},inplace=True)

In [ ]:
union=pib.merge(cov,on="Country")

In [ ]:
#Seleccionamos columnas relevantes
union=union[["Country","2020","continent","total_deaths_per_million"]]
union

### Correlación entre crecimiento del PIB y tasa de muertes por COVID-19 en 2020

In [ ]:
cc=union.corr().iloc[1,0] # Esto es para seleccionar una celda específica con el coef. corr.

In [ ]:
# Gráfico de calidad:
plt.style.use('seaborn')
fig, ax=plt.subplots(figsize=(7,6),dpi=80)
union.plot.scatter(x="total_deaths_per_million", y="2020", ax=ax)
ax.set_ylabel("Tasa variación del PIB en 2020  (%)",size=10)
ax.set_xlabel("Muertes por millón de hab. en 2020",size=10)
ax.set_title("Relación entre crecimiento del PIB y\n tasa de muertes por COVID-19 en 2020",
             size=13,pad=15)
ax.text(1200,3.2,"Coeficiente de\n corr.= "+str(round(cc,2)), ha="left",fontsize=10 ) 
fig.text(0.1,0.0,"Elaboración propia. Fuente: OWID y OCDE.", ha="left",fontsize=10 ) 
plt.show()

In [ ]:
europa=union[union.continent=="Europe"].copy()

In [ ]:
cc_e=europa.corr().iloc[1,0]

In [ ]:
# Gráfico de calidad:
plt.style.use('seaborn')
fig, ax=plt.subplots(figsize=(7,6),dpi=80)
europa.plot.scatter(x="total_deaths_per_million", y="2020", ax=ax)
ax.set_ylabel("Tasa variación del PIB en 2020  (%)",size=10)
ax.set_xlabel("Muertes por millón de hab. en 2020",size=10)
ax.set_title("Relación entre crecimiento del PIB y\n tasa de muertes por COVID-19 en 2020",
             size=13,pad=15)
ax.text(1200,3.2,"Coeficiente de\n corr.= "+str(round(cc_e,2)), ha="left",fontsize=10 ) 
fig.text(0.1,0.0,"Muestra: Países europeos de la OCDE.\nElaboración propia. Fuente: OWID y OCDE.", ha="left",fontsize=9 ) 
plt.show()